# 处理附件2中的数据

In [ ]:
import pandas as pd
import numpy as np
import os
from pprint import pprint
from scipy import interpolate

# 1 加载数据

In [4]:
B_hour_pre_raw = pd.read_excel('./data/附件2 监测点B、C空气质量预报基础数据.xlsx',sheet_name='监测点B逐小时污染物浓度与气象一次预报数据')
B_hour_actual_raw = pd.read_excel('./data/附件2 监测点B、C空气质量预报基础数据.xlsx',sheet_name='监测点B逐小时污染物浓度与气象实测数据')
C_hour_pre_raw = pd.read_excel('./data/附件2 监测点B、C空气质量预报基础数据.xlsx',sheet_name='监测点C逐小时污染物浓度与气象一次预报数据')
C_hour_actual_raw = pd.read_excel('./data/附件2 监测点B、C空气质量预报基础数据.xlsx',sheet_name='监测点C逐小时污染物浓度与气象实测数据')

B_day_actual_raw = pd.read_excel('./data/附件2 监测点B、C空气质量预报基础数据.xlsx',sheet_name='监测点B逐日污染物浓度实测数据')
C_day_actual_raw = pd.read_excel('./data/附件2 监测点B、C空气质量预报基础数据.xlsx',sheet_name='监测点C逐日污染物浓度实测数据')



In [5]:
B_hour_pre = B_hour_pre_raw
B_hour_actual = B_hour_actual_raw
C_hour_pre = C_hour_pre_raw
C_hour_actual = C_hour_actual_raw

B_day_actual = B_day_actual_raw
C_day_actual = C_day_actual_raw

In [7]:
B_hour_pre_raw.columns

Index(['模型运行日期', '预测时间', '地点', '近地2米温度（℃）', '地表温度（K）', '比湿（kg/kg）', '湿度（%）',
       '近地10米风速（m/s）', '近地10米风向（°）', '雨量（mm）', '云量', '边界层高度（m）', '大气压（Kpa）',
       '感热通量（W/m²）', '潜热通量（W/m²）', '长波辐射（W/m²）', '短波辐射（W/m²）', '地面太阳能辐射（W/m²）',
       'SO2小时平均浓度(μg/m³)', 'NO2小时平均浓度(μg/m³)', 'PM10小时平均浓度(μg/m³)',
       'PM2.5小时平均浓度(μg/m³)', 'O3小时平均浓度(μg/m³)', 'CO小时平均浓度(mg/m³)'],
      dtype='object')

In [8]:
B_hour_pre.set_axis(['start_time','time', 'place', 'temperature_2m','temperature_surface','specific_humidity','humidity','wind_speed_10m','wind_direction_10m','rainfall','cloudiness','height','pressure','sensible_heat','latent_heat','longwave','shortwave','solar_radiation','so2','no2','pm10', 'pm2.5', 'o3','co'], axis='columns', inplace=True)
B_hour_actual.set_axis(['time', 'place', 'so2','no2','pm10', 'pm2.5', 'o3','co','temperature', 'humidity', 'pressure','wind','direction'], axis='columns', inplace=True)

C_hour_pre.set_axis(['start_time','time', 'place', 'temperature_2m','temperature_surface','specific_humidity','humidity','wind_speed_10m','wind_direction_10m','rainfall','cloudiness','height','pressure','sensible_heat','latent_heat','longwave','shortwave','solar_radiation','so2','no2','pm10', 'pm2.5', 'o3','co'], axis='columns', inplace=True)
C_hour_actual.set_axis(['time', 'place', 'so2','no2','pm10', 'pm2.5', 'o3','co','temperature', 'humidity', 'pressure','wind','direction'], axis='columns', inplace=True)

B_day_actual.set_axis(['time', 'place', 'so2','no2','pm10', 'pm2.5', 'o3','co'], axis='columns', inplace=True)
C_day_actual.set_axis(['time', 'place', 'so2','no2','pm10', 'pm2.5', 'o3','co'], axis='columns', inplace=True)



B_day_actual = B_day_actual[0:-3]
C_day_actual = C_day_actual[0:-3]



In [9]:
# 暂时删除字符串的列，方便后续插值处理，后续会添加回去的
BHP_time = B_hour_pre['time']
BHA_time = B_hour_actual['time']
BDA_time = B_day_actual['time']

CHP_time = C_hour_pre['time']
CHA_time = C_hour_actual['time']
CDA_time = C_day_actual['time']
#C_hour_actual表的humidity这一列缺失值太多，直接舍弃

B_hour_pre = B_hour_pre.drop(columns=['place','time','start_time'])
B_hour_actual = B_hour_actual.drop(columns=['place','time'])
C_hour_pre = C_hour_pre.drop(columns=['place','time','start_time'])
C_hour_actual = C_hour_actual.drop(columns=['place','humidity','time'])
B_day_actual = B_day_actual.drop(columns=['place','time'])
C_day_actual = C_day_actual.drop(columns=['place','time'])

In [ ]:
C_hour_pre

In [10]:
# 缺失值统一化，把两种类型的缺失数据作为空值去处理
B_hour_pre = B_hour_pre.replace('—', np.nan).replace('NULL', np.nan).astype('float')
B_hour_actual = B_hour_actual.replace('—', np.nan).replace('NULL', np.nan).astype('float')  
C_hour_pre = C_hour_pre.replace('—', np.nan).replace('NULL', np.nan).astype('float')  
C_hour_actual = C_hour_actual.replace('—', np.nan).replace('NULL', np.nan).astype('float')  
B_day_actual = B_day_actual.replace('—', np.nan).replace('NULL', np.nan).astype('float')  
C_day_actual = C_day_actual.replace('—', np.nan).replace('NULL', np.nan).astype('float')  

In [14]:

# 异常值处理，小于0的都是异常值
B_hour_pre[ B_hour_pre< 0 ] = np.nan
B_hour_actual[ B_hour_actual< 0 ] = np.nan
C_hour_pre[ C_hour_pre< 0 ] = np.nan
C_hour_actual[ C_hour_actual< 0 ] = np.nan
B_day_actual[ B_day_actual< 0 ] = np.nan
C_day_actual[ C_day_actual< 0 ] = np.nan

B_hour_pre.insert(0, 'time', BHP_time)
B_hour_actual.insert(0, 'time', BHA_time)
C_hour_pre.insert(0, 'time', CHP_time)
C_hour_actual.insert(0, 'time', CHA_time)
B_day_actual.insert(0, 'time', BDA_time)
C_day_actual.insert(0, 'time', CHA_time)


# 2 插值

## 2.1 线性插值

In [ ]:
# filedict = {"BHP":B_hour_pre,"BHA":B_hour_actual,"CHP":C_hour_pre,"CHA":C_hour_actual,"BDA":B_day_actual,"CDA":C_day_actual}

filedict = {"BHP":B_hour_pre,"BHA":B_hour_actual,"CHP":C_hour_pre,"CHA":C_hour_actual,"BDA":B_day_actual,"CDA":C_day_actual}
for k,data in filedict.items():
    for indexs in data.columns:
        if indexs =='time':
            continue
        data['rownum'] = np.arange(data.shape[0])
        df_nona = data.dropna(subset = [indexs])
        f = interpolate.interp1d(df_nona['rownum'], df_nona[indexs])
        data[indexs] = f(data['rownum'])

    data = data.drop(columns=['rownum'])
    # 把地点加回去
    data['place'] = [k[0]]*len(data)
    if k =="BHP":
        data_BHP = data
    elif k =="BHA":
        data_BHA = data
    elif k =="CHP":
        data_CHP = data
    elif k =="CHA":
        data_CHA = data
    elif k =="BDA":
        data_BDA = data
    elif k =="CDA":
        data_CDA = data

In [ ]:
data_BHA

In [ ]:
writer=pd.ExcelWriter('data/data_2_linear.xlsx')   #定义writer
data_BHP.to_excel(writer,'监测点B逐小时污染物浓度与气象一次预报数据',index=False) # writer---writer；sheet名称---原始表（35698）
data_BHA.to_excel(writer,'监测点B逐小时污染物浓度与气象实测数据',index=False)
data_CHP.to_excel(writer,'监测点C逐小时污染物浓度与气象一次预报数据',index=False) # writer---writer；sheet名称---原始表（35698）
data_CHA.to_excel(writer,'监测点C逐小时污染物浓度与气象实测数据',index=False)
data_BDA.to_excel(writer,'监测点B逐日污染物浓度实测数据',index=False) # writer---writer；sheet名称---原始表（35698）
data_CDA.to_excel(writer,'监测点C逐日污染物浓度实测数据',index=False)
writer.save()

## 2.2 KNN插值

In [26]:
filedict = {"BHP":B_hour_pre,"BHA":B_hour_actual,"CHP":C_hour_pre,"CHA":C_hour_actual,"BDA":B_day_actual,"CDA":C_day_actual}

for k,data in filedict.items():
    def knn_mean(ts, n):
        out = np.copy(ts)
        for i, val in enumerate(ts):
            if np.isnan(val):
                n_by_2 = np.ceil(n/2)
                lower = np.max([0, int(i-n_by_2)])
                upper = np.min([len(ts)+1, int(i+n_by_2)])
                ts_near = np.concatenate([ts[lower:i], ts[i:upper]])
                out[i] = np.nanmean(ts_near)
        return out
    for indexs in data.columns:
        if indexs =='time':
            continue
        data[indexs] = knn_mean(data[indexs].values,24*3)
    if k =="BHP":
        data_BHP = data
    elif k =="BHA":
        data_BHA = data
    elif k =="CHP":
        data_CHP = data
    elif k =="CHA":
        data_CHA = data
    elif k =="BDA":
        data_BDA = data
    elif k =="CDA":
        data_CDA = data

In [30]:

# 判断是否有空值
print(data_CHA[data_CHA.isnull().T.any()])

Empty DataFrame
Columns: [time, so2, no2, pm10, pm2.5, o3, co, temperature, pressure, wind, direction]
Index: []


In [31]:
writer=pd.ExcelWriter('data/data_2_knn.xlsx')   #定义writer
data_BHP.to_excel(writer,'监测点B逐小时污染物浓度与气象一次预报数据',index=False) # writer---writer；sheet名称---原始表（35698）
data_BHA.to_excel(writer,'监测点B逐小时污染物浓度与气象实测数据',index=False)
data_CHP.to_excel(writer,'监测点C逐小时污染物浓度与气象一次预报数据',index=False) # writer---writer；sheet名称---原始表（35698）
data_CHA.to_excel(writer,'监测点C逐小时污染物浓度与气象实测数据',index=False)
data_BDA.to_excel(writer,'监测点B逐日污染物浓度实测数据',index=False) # writer---writer；sheet名称---原始表（35698）
data_CDA.to_excel(writer,'监测点C逐日污染物浓度实测数据',index=False)
writer.save()